In [1]:
import torch
import numpy as np
import sympy as sp
import os, sys
import symbolicregression
import requests
from IPython.display import display

/home/ziaeibideha/miniconda3/envs/md-bench-e2e/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "model.pt"
try:
    if not os.path.isfile(model_path):
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)

cuda:0
Model successfully loaded!


In [3]:
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=200,
                        n_trees_to_refine=100,
                        rescale=True
                        )

In [4]:
##Example of data

x = np.random.randn(10, 2)
# y = np.cos(2*np.pi*x[:,0])+x[:,1]**2
y = 2 * x[:, 0] + x[:, 1]


In [5]:
est.fit(x,y)
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
display(sp.parse_expr(model_str))

1.9885440717639546*x_0 + 0.9861907749585032*x_1 + 0.045117418131802306

In [6]:
est.predict(x)

array([ 4.43979676, -0.07754816,  1.66172747,  1.46604878,  1.86804931,
        1.33414029,  4.50396276,  0.20211903,  1.66520327,  1.03650048])

In [7]:
y

array([ 4.43770222, -0.12810275,  1.62564607,  1.43054235,  1.84529791,
        1.30212462,  4.50762506,  0.15909854,  1.61761641,  0.99871348])